Full name: Vũ Minh Phát

Student ID: 21127739

# HW2: Parallel Execution in CUDA

**To compile your file, you can use this command:** \
`nvcc tên-file.cu -o tên-file-chạy` \
***You can use Vietnamese to anwser the questions***

## Question 1A

Biên dịch file `HW2_P1.cu`:

In [1]:
!nvcc HW2_P1.cu -o HW2_P1

Chạy chương trình (để kích thước block mặc định):

In [3]:
!./HW2_P1

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
****************************

Input size: 16777217
Host time: 43.103168 ms

Kernel 1
Grid size: 16385, block size: 512
Kernel time = 2.185568 ms
CORRECT :)

Kernel 2
Grid size: 16385, block size: 512
Kernel time = 2.018048 ms
CORRECT :)

Kernel 3
Grid size: 16385, block size: 512
Kernel time = 1.002464 ms
CORRECT :)


**Câu hỏi**: So sánh "kernel time" của 3 hàm kernel với nhau và giải thích xem tại sao lại như vậy?

---

**Trả lời**: Từ kết quả chạy, với kích thước block mặc định là 512, ta thấy rằng:

- **Hàm kernel 1** có thời gian chạy là **hơn 2 ms** (kết quả của lần chạy gần nhất là 2.097632 ms).
- **Hàm kernel 2** có thời gian chạy là **khoảng 2 ms** (kết quả của lần chạy gần nhất là 2.000192 ms), nhanh hơn một chút so với hàm kernel 1.
- **Hàm kernel 3** có thời gian chạy là **khoảng 1 ms** (kết quả của lần chạy gần nhất là 0.978176 ms), nhanh hơn đáng kể so với cả hàm kernel 1 và 2.

**Giải thích**:

`1.` **Hàm kernel 1**: Interleaved Addressing

- Đây là phiên bản đơn giản của thuật toán "reduction", mỗi thread trong một block sẽ lần lượt thực hiện phép cộng giữa hai phần cách nhau một khoảng `stride` sau mỗi vòng lặp. Điều này gây ra tình trạng divergence (phân kỳ) cao giữa các warp khi chỉ một số thread trong mỗi warp thực hiện phép cộng tại mỗi bước `stride`.

- Ngoài ra, cách sử dụng bộ nhớ ("global memory") không liên tục có thể gây ra các vấn đề về *coalescing* (liên kết truy cập bộ nhớ), làm tăng độ trễ truy cập bộ nhớ. Vì việc truy xuất các vị trí liên tiếp trên bộ nhớ (*burst*) sẽ giúp giảm thời gian các thread truy xuất đến bộ nhớ để lấy dữ liệu.

- Đồng thời, để xác định các thread cần thực hiện phép cộng ở mỗi bước `stride`, ta cần sử dụng toán tử `%` (modulo) và đây là một phép toán tốn nhiều thời gian để hoàn thành.

- Điều này dẫn đến việc hàm kernel 1 có thời gian chạy lâu nhất trong ba phiên bản của hàm kernel.

`2.` **Hàm kernel 2**: Reduced Warp Divergence

- Hàm kernel 2 đã tối ưu hơn bằng cách giảm số warp phân kỳ. Các thread được sắp xếp sao cho chỉ các thread cần thiết trong block được kích hoạt ở mỗi bước `stride` và đó là các thread liền kề đầu tiên.

- Điều này làm giảm tình trạng phân kỳ warp và giảm thời gian chờ đợi khi chỉ một vài warp đầu tiên trong block thực hiện phép cộng tại mỗi bước `stride`. Thông qua đó, hàm kernel 2 giúp tăng hiệu suất hoạt động của GPU và có thời gian thực thi ngắn hơn một chút so với hàm kernel 1.

- Mặc dù tình trạng phân kỳ warp đã giảm, nhưng sau mỗi bước `stride`, các thread liền kề nhau phải truy cập đến các vị trí không liên tục trên bộ nhớ, không tận dụng tối đa cơ chế liên kết bộ nhớ (coalescing). Vì vậy, hiệu suất của hàm kernel 2 chưa thực sự nổi bật so với hàm kernel 1.

`3.` **Hàm kernel 3**: Reduced Warp Divergence + Sequential Addressing

- Hàm kernel 3 kết hợp hai cải tiến: giảm phân kỳ trong warp và sử dụng "sequential addressing", trong đó các thread thực hiện tính tổng giữa các phần tử gần kề, thay vì cách xa nhau, và khoảng cách càng giảm dần sau mỗi bước `stride`. Điều này giúp giảm thiểu số lượng warp phân kỳ và tận dụng hiệu quả cơ chế liên kết bộ nhớ (coalescing) khi các thread liền kề trong warp truy cập đến các vị trí liên tục trên bộ nhớ. Nhờ đó, hàm kernel này giảm bớt chi phí truy cập vào global memory và tăng hiệu suất tổng thể của chương trình.

- Vì hàm kernel 3 là hàm được tối ưu nhất, nên không quá khó hiểu khi thời gian thực thi của nó là ngắn nhất trong cả ba phiên bản (nhanh hơn gấp đôi so với hàm kernel 1 và 2).

**Tổng kết**: Hàm kernel 3 nhanh nhất vì kết hợp cách truy cập bộ nhớ hiệu quả và giảm thiểu số lượng warp bị phân kỳ, trong khi hàm kernel 1 và 2 ít tối ưu hơn do các phương pháp truy cập bộ nhớ không hiệu quả và tình trạng phân kỳ warp thường xuyên xảy ra.

> References:  
> [1] Slide "Optimizing Parallel Reduction in CUDA" by Mark Harris (NVIDIA Developer Technology) - [developer.download.nvidia.com](https://developer.download.nvidia.com/assets/cuda/files/reduction.pdf).  
> [2] Book "Programming Massively Parallel Processors: A Hands-on Approach", chapter 5 "Memory architecture and data locality" by David Kirk and Wen-mei Hwu.


Dùng `nvprof` để xem chi tiết thực thi của chương trình:

In [4]:
!nvprof ./HW2_P1

==1203== NVPROF is profiling process 1203, command: ./HW2_P1
**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
****************************

Input size: 16777217
Host time: 51.849247 ms

Kernel 1
Grid size: 16385, block size: 512
Kernel time = 2.320576 ms
CORRECT :)

Kernel 2
Grid size: 16385, block size: 512
Kernel time = 2.457312 ms
CORRECT :)

Kernel 3
Grid size: 16385, block size: 512
Kernel time = 1.053312 ms
CORRECT :)
==1203== Profiling application: ./HW2_P1
==1203== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   92.46%  60.185ms         3  20.062ms  16.513ms  22.082ms  [CUDA memcpy HtoD]
                    3.08%  2.0051ms         1  2.0051ms  2.0051ms  2.0051ms  reduceBlksKernel2(int*, int*, int)
                    2.93%  1.9090ms         1  1.9090ms  1.9090ms  1.9090ms  reduceBlksKernel1(int*, int*

**Nhận xét về thời gian thực thi các công việc trong phần "GPU activities"**:

Dựa vào kết quả chi tiết của `nvprof`, ta có một vài nhận xét về thời gian thực thi của các công việc trong phần "GPU activities" như sau:

`1.` **[CUDA memcpy HtoD] ~ 90%**:

- Phần lớn thời gian hoạt động của GPU (khoảng 90%) được dành cho việc sao chép dữ liệu từ Host (CPU) sang Device (GPU), với tổng cộng 3 lần gọi.

- Với mỗi lần gọi, ta đã thực hiện sao chép $2^{24} + 1$ phần tử có kiểu dữ liệu `int` từ CPU sang GPU, đây là một lượng dữ liệu tương đối lớn. Điều này phần nào lý giải cho việc thời gian sao chép dữ liệu để chuẩn bị chạy hàm kernel lại chiếm phần lớn thời gian hoạt động của GPU.

- Đây sẽ là một yếu tố mà chúng ta cần xem xét để có thể tối ưu hóa hiệu suất của chương trình. Ví dụ như giảm lượng dữ liệu cần truyền đi hoặc tận dụng các loại bộ nhớ khác trên GPU mà có thời gian truy cập nhanh hơn.

`2.` **reduceBlksKernel1 ~ 4%**:

- Thời gian thực thi của hàm kernel `reduceBlksKernel1` chiếm khoảng 4% tổng thời gian hoạt động của GPU. Con số này nhỏ hơn nhiều so với thời gian sao chép dữ liệu từ host sang device.

- Đây là phiên bản đầu tiên, đơn giản nhất trong số ba hàm kernel mà ta đã cài đặt. Việc truy cập bộ nhớ không hiệu quả và tình trạng phân kỳ warp cao khiến thời gian thực thi của nó tương đối dài so với phiên bản tối ưu hơn.

`3.` **reduceBlksKernel2 ~ 4%**:

- Tương tự như hàm kernel 1, thời gian thực thi của hàm kernel `reduceBlksKernel2` chiếm khoảng 4% tổng thời gian hoạt động của GPU. Và con số này cũng nhỏ hơn nhiều so với thời gian sao chép dữ liệu từ host sang device.

- Mặc dù đã được tối ưu hóa bằng cách giảm thiểu số lượng warp bị phân kỳ trong quá trình "reduction" ở mỗi block, nhưng hàm kernel này vẫn sử dụng cách truy cập bộ nhớ không hiệu quả. Có vẻ như tình trạng "interleaved addressing" là một rào cản lớn khiến thời gian thực thi của hàm kernel này không thay đổi nhiều so với phiên bản đầu tiên.

- Do đó, ta cần tìm giải pháp để tiếp tục tối ưu hóa hàm kernel này nhằm tăng tốc độ thực thi. Đặc biệt là ở cách tổ chức các thread trong mỗi block để có thể giảm thiểu chi phí khi truy cập global memory.

`4.` **reduceBlksKernel3 ~ 2%**:

- `reduceBlksKernel3` là hàm kernel tốt nhất trong ba phiên bản mà ta đã cài đặt. Thời gian thực thi của nó chỉ chiếm khoảng 2% tổng thời gian hoạt động của GPU, nhanh hơn gấp đôi so với hàm kernel 1 và 2.

- Kết quả này cho thấy hàm kernel 3 có hiệu suất cao hơn đáng kể so với hai hàm kernel trước đó nhờ vào cách tổ chức các thread hiệu quả để có thể tận dụng cơ chế liên kết bộ nhớ (coalescing) và giảm thiểu tình trạng phân kỳ warp.

`5.` **[CUDA memcpy DtoH] ~ 0.01%**:

- Việc sao chép dữ liệu từ Device (GPU) về Host (CPU) tuy được thực hiện 3 lần (bằng với số lần sao chép từ Host sang Device) nhưng chiếm tỷ lệ thời gian rất nhỏ so với thời gian sao chép từ Host sang Device.

- Điều này cũng dễ hiểu vì dữ liệu được truyền từ GPU về CPU chỉ là một số nguyên thể hiện kết quả cuối cùng của phép tính, kích thước nhỏ hơn rất nhiều so với dữ liệu đầu vào. Do đó, thời gian sao chép từ GPU về CPU không ảnh hưởng nhiều đến tổng thời gian hoạt động của GPU.

- Như vậy thời gian sao chép dữ liệu từ GPU về CPU không phải là nút thắt hiệu năng trong chương trình.

**Kết luận**:

- Kết quả trên chỉ ra rằng việc truyền dữ liệu từ Host sang Device là bước tốn thời gian nhất, chiếm khoảng 90% thời gian hoạt động của GPU. Để cải thiện hiệu năng tổng thể, ta có thể xem xét giảm bớt lượng dữ liệu cần truyền hoặc thực hiện nhiều phép tính trên GPU để tận dụng tốt hơn tài nguyên của GPU.

- Trong ba hàm kernel, hàm `reduceBlksKernel3` có hiệu suất tốt nhất với thời gian thực thi ngắn hơn gấp đôi so với hai hàm kernel còn lại. Điều này cho thấy rằng việc tối ưu hóa cách tổ chức các thread trong mỗi block và cách truy cập bộ nhớ có thể giúp tăng hiệu suất của chương trình lên đáng kể.

> References:  
> [1] Blog "Profiling a CUDA application with nvprof" by Talles L - [dev.to](https://dev.to/tallesl/profiling-a-cuda-application-with-nvprof-56op).  

## Question 1B


Chạy chương trình với các kích thước block khác nhau:

- Với kích thước block: `1024`

In [5]:
!./HW2_P1 1024

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
****************************

Input size: 16777217
Host time: 46.211166 ms

Kernel 1
Grid size: 8193, block size: 1024
Kernel time = 2.906656 ms
CORRECT :)

Kernel 2
Grid size: 8193, block size: 1024
Kernel time = 2.726048 ms
CORRECT :)

Kernel 3
Grid size: 8193, block size: 1024
Kernel time = 1.269728 ms
CORRECT :)


- Với kích thước block: `512`

In [6]:
!./HW2_P1 512

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
****************************

Input size: 16777217
Host time: 45.076481 ms

Kernel 1
Grid size: 16385, block size: 512
Kernel time = 2.185280 ms
CORRECT :)

Kernel 2
Grid size: 16385, block size: 512
Kernel time = 2.033856 ms
CORRECT :)

Kernel 3
Grid size: 16385, block size: 512
Kernel time = 1.008352 ms
CORRECT :)


- Với kích thước block: `256`

In [7]:
!./HW2_P1 256

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
****************************

Input size: 16777217
Host time: 43.702271 ms

Kernel 1
Grid size: 32769, block size: 256
Kernel time = 1.867840 ms
CORRECT :)

Kernel 2
Grid size: 32769, block size: 256
Kernel time = 1.655456 ms
CORRECT :)

Kernel 3
Grid size: 32769, block size: 256
Kernel time = 0.875392 ms
CORRECT :)


- Với kích thước block: `128`

In [8]:
!./HW2_P1 128

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
****************************

Input size: 16777217
Host time: 45.043362 ms

Kernel 1
Grid size: 65537, block size: 128
Kernel time = 1.649824 ms
CORRECT :)

Kernel 2
Grid size: 65537, block size: 128
Kernel time = 1.435744 ms
CORRECT :)

Kernel 3
Grid size: 65537, block size: 128
Kernel time = 0.787488 ms
CORRECT :)


Block size | Grid size | Num blocks / SM | Occupancy (%) | Kernel time (ms)
--- | --- | --- | --- | ---
1024 | 8193 | 1 | 100 | 2.885760
512 | 16385 | 2 | 100 | 2.130336
256 | 32769 | 4 | 100 | 1.784672
128 | 65537 | 8 | 100 | 1.641888

**Giải thích về kết quả của hàm kernel 1 theo mẫu bảng biểu bên trên**:

**Phân tích**:

Từ bảng "[Technical Specifications per Compute Capability](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#features-and-technical-specifications-technical-specifications-per-compute-capability)" của NVIDIA, với GPU có Compute Capability là `7.5`, mỗi SM có:
- Maximum number of resident blocks per SM: 16. (Số block tối đa có thể chạy đồng thời trên mỗi SM.)

- Maximum number of resident threads per SM: 1024. (Số thread tối đa có thể chạy đồng thời trên mỗi SM.)

**Cách tính giá trị cho mỗi cột**:

`0.` Với hai cột `Grid size` và `Kernel time (ms)`:

- Khi ta chạy chương trình với các kích thước block (`Block size`) khác nhau, thì thông tin về `Grid size` và `Kernel time (ms)` của hàm kernel 1 đã được hiển thị rõ ràng trong kết quả chạy. Do đó, ta có thể lấy trực tiếp giá trị từ kết quả chạy để điền vào bảng.

`1.` Với cột `Num blocks / SM`:

- Để tính số lượng block thực tế chạy trên mỗi SM, ta chia *số lượng thread tối đa mà mỗi SM có thể chứa* (`1024`) cho *số lượng thread trên mỗi block* (`Block size`):

$$\left\lfloor \frac{\text{Num threads / SM}}{\text{Num threads / block}} \right\rfloor = \text{Num blocks / SM}$$

- Tuy nhiên, con số này chưa phải kết quả cuối cùng. Theo ràng buộc về *số lượng block tối đa mà mỗi SM có thể chứa*, nếu kết quả của phép tính bên trên lớn hơn `16`, ta sẽ giới hạn kết quả ở giá trị `16`.

`2.` Với cột `Occupancy (%)`:

- Tỷ lệ sử dụng (*occupancy*) được xác định bằng cách so sánh *số lượng thread thực tế chạy trên mỗi SM* với *số lượng thread tối đa mà mỗi SM có thể chứa* (`1024`):

$$\text{Occupancy (\%)} = \frac{\text{Num threads SM contains}}{\text{Max num threads SM can contain}} \times 100\%$$

- Trong đó:
    - *Số lượng thread tối đa mà mỗi SM có thể chứa* đã được chúng ta xác định trước đó là `1024`.

    - *Số lượng thread thực tế chạy trên mỗi SM* chính là *số lượng block thực tế chạy trên mỗi SM* (`Num blocks / SM`) nhân với *số lượng thread trên mỗi block* (`Block size`).

**Bảng tính toán chi tiết**:

`1.` Với **Block size** = `1024`:

- Số lượng block thực tế chạy trên mỗi SM là:
$$\text{Num blocks / SM} = \left\lfloor \frac{1024}{1024} \right\rfloor = 1 \leq 16$$

- Vì số block thực tế chạy trên mỗi SM không vượt quá 16, nên ta có:
$$\text{Num blocks / SM} = 1$$

- Số lượng thread thực tế chạy trên mỗi SM là:
$$\text{Num threads SM contains} = 1 \times 1024 = 1024$$

- Tỷ lệ sử dụng trong trường hợp này là:
$$\text{Occupancy (\%)} = \frac{1024}{1024} \times 100\% = 100\%$$

`2.` Với **Block size** = `512`:

- Số lượng block thực tế chạy trên mỗi SM là:
$$\text{Num blocks / SM} = \left\lfloor \frac{1024}{512} \right\rfloor = 2 \leq 16$$

- Vì số block thực tế chạy trên mỗi SM không vượt quá 16, nên ta có:
$$\text{Num blocks / SM} = 2$$

- Số lượng thread thực tế chạy trên mỗi SM là:
$$\text{Num threads SM contains} = 2 \times 512 = 1024$$

- Tỷ lệ sử dụng trong trường hợp này là:
$$\text{Occupancy (\%)} = \frac{1024}{1024} \times 100\% = 100\%$$

`3.` Với **Block size** = `256`:

- Số lượng block thực tế chạy trên mỗi SM là:
$$\text{Num blocks / SM} = \left\lfloor \frac{1024}{256} \right\rfloor = 4 \leq 16$$

- Vì số block thực tế chạy trên mỗi SM không vượt quá 16, nên ta có:
$$\text{Num blocks / SM} = 4$$

- Số lượng thread thực tế chạy trên mỗi SM là:
$$\text{Num threads SM contains} = 4 \times 256 = 1024$$

- Tỷ lệ sử dụng trong trường hợp này là:
$$\text{Occupancy (\%)} = \frac{1024}{1024} \times 100\% = 100\%$$

`4.` Với **Block size** = `128`:

- Số lượng block thực tế chạy trên mỗi SM là:
$$\text{Num blocks / SM} = \left\lfloor \frac{1024}{128} \right\rfloor = 8 \leq 16$$

- Vì số block thực tế chạy trên mỗi SM không vượt quá 16, nên ta có:
$$\text{Num blocks / SM} = 8$$

- Số lượng thread thực tế chạy trên mỗi SM là:
$$\text{Num threads SM contains} = 8 \times 128 = 1024$$

- Tỷ lệ sử dụng trong trường hợp này là:
$$\text{Occupancy (\%)} = \frac{1024}{1024} \times 100\% = 100\%$$


**Câu hỏi**: Suy nghĩ và giải thích xem tại sao khi thay đổi block size thì "kernel time" lại thay đổi như vậy?

---

**Trả lời**: Dựa vào bảng biểu đã điền, ta thấy rằng mặc dù `Occupancy (%)` đều đạt 100% ở tất cả các trường hợp, nhưng `Kernel time (ms)` lại giảm dần khi giảm `block size` từ 1024 xuống 128. Điều này có thể đến từ một số nguyên nhân sau:

`1.` **Hiệu quả của việc sử dụng tài nguyên trên mỗi SM**:

- Với các `block size` lớn (như 1024), mỗi block chứa số lượng lớn thread. Khi chỉ có một (ít) block được chạy trên mỗi SM, chỉ có một lượng lớn tài nguyên được sử dụng bởi block đó. Điều này có thể dẫn đến việc không tận dụng được hết khả năng song song của các SM, vì mỗi block duy nhất sẽ chiếm hết các tài nguyên của SM.

- Khi giảm `block size` xuống, số lượng block chạy trên mỗi SM tăng lên (như ở đây, từ 1 block khi `block size` là 1024 đến 8 block khi `block size` là 128). Điều này cho phép nhiều block được thực thi song song hơn trong một SM, dẫn đến mức độ song song cao hơn và sử dụng tài nguyên hiệu quả hơn. Việc có nhiều block giúp giảm thời gian chờ của các thread, tận dụng các đơn vị tính toán của SM tốt hơn, từ đó giảm tổng thời gian thực thi của hàm kernel.

`2.` **Hiệu quả khi có nhiều warp được kích hoạt đồng thời**:

- Trong GPU, các thread được xử lý theo các "warp" (gồm 32 thread liên tiếp). Khi `block size` lớn, số lượng warp trên mỗi block tăng, nhưng đồng thời giới hạn tổng số block có thể chạy song song.

- Khi giảm `block size`, mỗi block có ít warp hơn, nhưng số block trên mỗi SM tăng lên, cho phép kích hoạt nhiều warp từ các block khác nhau. Điều này giúp tăng khả năng xử lý song song, giảm thời gian chờ đợi giữa các warp, và dẫn đến giảm thời gian thực thi tổng thể.

`3.` **Lợi ích từ việc che dấu độ trễ bộ nhớ (Memory Latency Hiding)**:

- Các GPU thường gặp vấn đề về độ trễ truy cập bộ nhớ toàn cục (global memory latency). Khi `block size` nhỏ hơn, nhiều block chạy đồng thời giúp tăng số lượng warp đang chờ và thực thi, điều này giúp che giấu độ trễ khi truy cập bộ nhớ toàn cục vì các warp khác có thể tiếp tục thực thi trong khi một warp đang chờ dữ liệu được đọc từ bộ nhớ.

- Do đó, giảm `block size` giúp tận dụng khả năng ẩn độ trễ bộ nhớ hiệu quả hơn, nhờ vào việc có thêm nhiều warp đang trong tình trạng sẵn sàng hoạt động.

**Kết luận**:

- Nhìn chung, khi `block size` giảm, số lượng block tăng, cho phép GPU tận dụng tốt hơn khả năng song song của các SM và giảm thời gian thực thi của hàm kernel. 

> References:  
> [1] Topic "The relationship between the execution time and blocksize" on [forums.developer.nvidia.com](https://forums.developer.nvidia.com/t/the-relationship-between-the-execution-time-and-blocksize/51318).  
> [2] Question "Increasing block size decreases performance" on [stackoverflow.com](https://stackoverflow.com/questions/5048866/increasing-block-size-decreases-performance).  

## Question 2A

Biên dịch file `HW2_P2.cu`:

In [9]:
!nvcc HW2_P2.cu -o HW2_P2

Chạy chương trình (để kích thước block mặc định):

In [11]:
!./HW2_P2

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
****************************

Processing time (use host): 4433.424805 ms

Basic Matrix Multiplication:
Grid size: 32 * 32, block size: 32 * 32
Processing time (use device): 8.483232 ms
Error between device result and host result: 0.000004

Shared memory Matrix Multiplication:
Grid size: 32 * 32, block size: 32 * 32
Processing time (use device): 5.715968 ms
Error between device result and host result: 0.000004

Dùng `nvprof` để xem chi tiết thực thi của chương trình:

In [12]:
!nvprof ./HW2_P2

==1382== NVPROF is profiling process 1382, command: ./HW2_P2
**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
****************************

Processing time (use host): 3625.827393 ms

Basic Matrix Multiplication:
Grid size: 32 * 32, block size: 32 * 32
Processing time (use device): 8.730368 ms
Error between device result and host result: 0.000004

Shared memory Matrix Multiplication:
Grid size: 32 * 32, block size: 32 * 32
Processing time (use device): 5.659264 ms
Error between device result and host result: 0.000004==1382== Profiling application: ./HW2_P2
==1382== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   33.97%  3.2661ms         1  3.2661ms  3.2661ms  3.2661ms  matrix_multiplication_kernel1(float*, float*, float*, int, int, int)
                   27.61%  2.6544ms         1  2.6544ms  2.6544ms  2.6544

**Nhận xét về thời gian thực thi các công việc trong phần "GPU activities"**:

Dựa vào kết quả chi tiết của `nvprof`, ta có một vài nhận xét về thời gian thực thi của các công việc trong phần "GPU activities" như sau:

`1.` **matrix_multiplication_kernel1 ~ 35%**:

- Hàm kernel thứ nhất, `matrix_multiplication_kernel1`, sử dụng global memory mà không tối ưu hóa qua shared memory, chiếm khoảng 35% tổng thời gian hoạt động của GPU.

- Điều này là do mỗi thread phải truy cập nhiều lần vào global memory lấy dữ liệu để thực hiện phép nhân và cộng, gây ra độ trễ lớn. Tuy nhiên, thời gian thực thi của hàm kernel này vẫn ngắn hơn đáng kể so với thời gian thực thi trên CPU.

`2.` **matrix_multiplication_kernel2 ~ 30%**:

- Hàm kernel thứ hai, `matrix_multiplication_kernel2`, sử dụng shared memory để lưu trữ các tile của ma trận $A$ và $B$, giúp giảm thiểu số lần truy cập vào global memory, chiếm khoảng 30% tổng thời gian hoạt động của GPU.

- Thời gian thực thi của hàm `*_kernel2` ngắn hơn hàm `*_kernel1` do hiệu quả của việc sử dụng shared memory giúp giảm thiểu độ trễ khi cần truy cập dữ liệu từ bộ nhớ. Điều này cho thấy rằng việc tối ưu hóa cách truy cập bộ nhớ có thể giúp tăng hiệu suất của chương trình trên GPU.

- Tuy nhiên, do kích thước dữ liệu là khá lớn và việc đồng bộ giữa các thread, nên ta thấy thời gian thực thi của hàm kernel này vẫn chiếm một phần lớn trong tổng thời gian hoạt động của GPU.

`3.` **[CUDA memcpy DtoH] ~ 25%**:

- Hai lần `CUDA memcpy DtoH` (Device to Host), chiếm khoảng 25% tổng thời gian hoạt động của GPU.

- Điều này là bình thường vì truyền dữ liệu từ GPU về CPU thường tốn kém thời gian, đặc biệt là khi lượng dữ liệu lớn (ma trận kết quả $C$ có kích thước $2^{10} \times 2^{10}$).

`4.` **[CUDA memcpy HtoD] ~ 10%**:

- Bốn lần `CUDA memcpy HtoD` (Host to Device), chiếm khoảng 10% tổng thời gian hoạt động của GPU.

- Ở đây ta có một quan sát thú vị: Mỗi khi một hàm kernel được gọi, *lượng dữ liệu đầu vào* cần được sao chép từ host sang device (2 ma trận $A$ và $B$, mỗi ma trận có $2^{19}$ phần tử) sẽ bằng với *lượng dữ liệu đầu ra* cần được sao chép từ device về host (1 ma trận $C$ có $2^{20}$ phần tử). Về lý thuyết, số lượng byte cần sao chép ở hai giai đoạn này sẽ bằng nhau, nhưng thực tế lại cho thấy rằng: thời gian thực thi của 2 lần `HtoD` ngắn hơn gấp đôi so với 1 lần `DtoH`. Điều này có thể liên quan đến cơ chế hoạt động bên dưới của GPU và ta có thể tìm hiểu sâu hơn về điều này để tối ưu hóa chương trình hơn nữa.

**Kết luận**:

- Các hàm kernel `matrix_multiplication_kernel1` và `matrix_multiplication_kernel2` là phần tiêu tốn nhiều thời gian nhất trên GPU, chiếm khoảng 65% tổng thời gian. Sự khác biệt về thời gian thực thi giữa hai hàm kernel là khá đáng kể, điều này cho thấy hiệu quả của việc sử dụng shared memory trong hàm `*_kernel2`.

- Quá trình truyền dữ liệu giữa host và device (cả `HtoD` và `DtoH`) cũng chiếm một phần đáng kể thời gian, lên tới khoảng 35%. Điều này nhấn mạnh rằng việc tối ưu hóa phép nhân ma trận trên GPU không chỉ cần tập trung vào thuật toán của hàm kernel, mà còn phải xem xét đến chi phí truyền dữ liệu giữa host và device.


## Question 2B

**For Basic Matrix Multipication**

1. How many floating operations are being performed in your matrix multiply
kernel? Explain

- Dựa vào khung chương trình trong file `HW2_P2.cu`, ta biết rằng:
    - Ma trận $A$ có $m$ hàng và $n$ cột.
    - Ma trận $B$ có $n$ hàng và $k$ cột.
    - Ma trận $C$ có $m$ hàng và $k$ cột.
    - Với: $m=2^{10}$, $n=2^{9}$, $k=2^{10}$.

- **Giải thích**:
    - Trong hàm `*_kernel1`, đối với mỗi phần tử $C_{ij}$ của ma trận kết quả $C$, hàm kernel tính tích vô hướng giữa hàng thứ $i$ của ma trận $A$ với cột thứ $j$ của ma trận $B$.
    
    - Để tính được giá trị của $C_{ij}$, ta cần thực hiện $n$ phép nhân và $n-1 \approx n$ phép cộng (với $n$ là số cột của ma trận $A$ và số hàng của ma trận $B$).

    - Về mặt lý thuyết thì ta chỉ cần thực hiện $n - 1$ phép cộng, nhưng trong trường hợp này thì ta sẽ sử dụng $n$ phép cộng (tức là thêm một phép cộng so với lý thuyết) để đơn giản hóa quá trình cài đặt chương trình.

    - Do đó, để tính được giá trị cho mỗi phần tử $C_{ij}$, ta cần thực hiện $n$ phép nhân và $n$ phép cộng, tổng cộng là $2n$ phép toán trên số chấm động.

- **Tổng số phép toán trên số chấm động (floating-point operations)**: Vì ma trận $C$ có $m \times k$ phần tử, nên tổng số phép toán trên số chấm động cần thực hiện trong hàm kernel là:
$$
\begin{split}
\text{Total FLOPs} &= (m \times k) \times 2n
\end{split}
$$

- Trong trường hợp cụ thể của bài tập này, tổng số phép toán trên số chấm động cần thực hiện trong hàm kernel là:
$$
\begin{split}
\text{Total FLOPs}
&= (2^{10} \times 2^{10}) \times 2 \times 2^{9} \\
&= 2^{30}
\end{split}
$$

2. How many global memory reads are being performed by your kernel? Explain

- **Giải thích**:
    - Trong hàm `*_kernel1`, mỗi thread cần đọc một hàng từ ma trận $A$ (gồm $n$ phần tử) và một cột từ ma trận $B$ (gồm $n$ phần tử) để phục vụ cho việc tính toán. Do đó, tổng số lần đọc từ *global memory* sẽ là $2n$ lần đọc cho mỗi thread.

- **Tổng số lần đọc từ global memory**: Với mỗi phần tử $C_{ij}$ của ma trận kết quả $C$, ta cần thực hiện $2n$ lần đọc từ *global memory*. Vì ma trận $C$ có $m \times k$ phần tử, nên tổng số lần đọc từ *global memory* cần thực hiện trong hàm kernel là:
$$
\begin{split}
\text{Total Global Memory Reads} &= (m \times k) \times 2n
\end{split}
$$

- Trong trường hợp cụ thể của bài tập này, tổng số lần đọc từ *global memory* cần thực hiện trong hàm kernel là:
$$
\begin{split}
\text{Total Global Memory Reads}
&= (2^{10} \times 2^{10}) \times 2 \times 2^{9} \\
&= 2^{30}
\end{split}
$$

3. How many global memory writes are being performed by your kernel? Explain.

- **Giải thích**:
    - Trong hàm `*_kernel1`, mỗi thread cần ghi một phần tử sau khi được tính toán vào ma trận $C$. Do đó, với mỗi phần tử $C_{ij}$ của ma trận kết quả $C$, ta cần thực hiện một lần ghi vào *global memory*.

- **Tổng số lần ghi vào global memory**: Vì ma trận $C$ có $m \times k$ phần tử, nên tổng số lần ghi vào *global memory* cần thực hiện trong hàm kernel là:
$$
\begin{split}
\text{Total Global Memory Writes} &= m \times k
\end{split}
$$

- Trong trường hợp cụ thể của bài tập này, tổng số lần ghi vào *global memory* cần thực hiện trong hàm kernel là:
$$
\begin{split}
\text{Total Global Memory Writes}
&= 2^{10} \times 2^{10} \\
&= 2^{20}
\end{split}
$$

**For Tiled Matrix Multipication**

1. How many floating operations are being performed in your matrix multiply
kernel? Explain

- Dựa vào khung chương trình trong file `HW2_P2.cu`, ta biết rằng:
    - Mỗi tile có kích thước $TILE\_WIDTH \times TILE\_WIDTH$.
    - Với: $TILE\_WIDTH = 32$.

- **Giải thích**:
    - Trên thực tế, hàm `*_kernel2` (**tiled kernel**) vẫn thực hiện thuật toán nhân hai ma trận tương tự như hàm `*_kernel1`, nhưng được tiến hành trên từng tile nhỏ hơn của ma trận $C$.

    - Do đó, như đã giải thích ở trên, để tính được giá trị cho mỗi phần tử $C_{ij}$ trong ma trận kết quả $C$, ta cần thực hiện $2n$ phép toán trên số chấm động, gồm $n$ phép nhân và $n$ phép cộng.

- **Tổng số phép toán trên số chấm động (floating-point operations)**: Vì ma trận $C$ có $m \times k$ phần tử, nên tổng số phép toán trên số chấm động cần thực hiện trong hàm kernel là:
$$
\begin{split}
\text{Total FLOPs} &= (m \times k) \times 2n
\end{split}
$$

- Trong trường hợp cụ thể của bài tập này, tổng số phép toán trên số chấm động cần thực hiện trong hàm kernel là:
$$
\begin{split}
\text{Total FLOPs}
&= (2^{10} \times 2^{10}) \times 2 \times 2^{9} \\
&= 2^{30}
\end{split}
$$

2. How many global memory reads are being performed by your kernel? Explain

- **Giải thích**:
    - Trong hàm `*_kernel2`, mỗi tile của ma trận $A$ và ma trận $B$ được đọc một lần từ *global memory* và lưu vào *shared memory* tại mỗi lần lặp. Điều này giúp giảm số lần truy cập vào *global memory* (thường là chậm hơn) và tăng hiệu suất tổng thể của hàm kernel.

    - Đối với mỗi (thread-)block xử lý một tile có kích thước $TILE\_WIDTH \times TILE\_WIDTH$ của ma trận $C$, chúng ta cần đọc một tile có kích thước $TILE\_WIDTH \times TILE\_WIDTH$ từ ma trận $A$ và một tile khác có kích thước tương tự từ ma trận $B$.

- **Tổng số lần đọc từ global memory**: Tổng số lần đọc từ *global memory* trong hàm `*_kernel2` (tiled kernel) được giảm đi theo hệ số kích thước của tile (ở đây là $TILE\_WIDTH = 32$) so với hàm `*_kernel1` (basic kernel). Nếu $n$ chia hết cho $TILE\_WIDTH$, thì tổng số lần đọc từ *global memory* sẽ xấp xỉ:
$$
\begin{split}
\text{Total Global Memory Reads} &\approx \frac{(m \times k) \times 2n}{TILE\_WIDTH}
\end{split}
$$

- Trong trường hợp cụ thể của bài tập này, tổng số lần đọc từ *global memory* cần thực hiện trong hàm kernel là:
$$
\begin{split}
\text{Total Global Memory Reads}
&= \frac{(2^{10} \times 2^{10}) \times 2 \times 2^{9}}{32} \\
&= 2^{25}
\end{split}
$$

3. How many global memory writes are being performed by your kernel? Explain

- **Giải thích**:
    - Cũng giống như hàm `*_kernel1` (basic kernel), trong hàm `*_kernel2` (tiled kernel), mỗi thread cần ghi một phần tử sau khi được tính toán vào ma trận $C$. Do đó, với mỗi phần tử $C_{ij}$ của ma trận kết quả $C$, ta cần thực hiện một lần ghi vào *global memory*.

- **Tổng số lần ghi vào global memory**: Vì ma trận $C$ có $m \times k$ phần tử, nên tổng số lần ghi vào *global memory* cần thực hiện trong hàm kernel là:
$$
\begin{split}
\text{Total Global Memory Writes} &= m \times k
\end{split}
$$

- Trong trường hợp cụ thể của bài tập này, tổng số lần ghi vào *global memory* cần thực hiện trong hàm kernel là:
$$
\begin{split}
\text{Total Global Memory Writes}
&= 2^{10} \times 2^{10} \\
&= 2^{20}
\end{split}
$$

> References:  
> [1] GitHub repository "matrix-cuda" by Zhengchun Liu - [github.com](https://github.com/lzhengchun/matrix-cuda).  